Proiect IC : Symmetric-key Corruption Detection : When XOR-MACs Meet Combinatorial Group Testing 

Lidia Istrate 342C4

XOR-GTM with t x m test matrix

In [2]:
def matrix_to_vector_power(H, R):
    t = len(H)
    m = len(H[0])
    v = len(R)
    
    result = []
    for i in range (0, v):
        result.append([])
        for j in range(0, m):
            if len(R[i]) == 1:
                result[i].append(H[R[i][0] - 1][j])
            else:
                xor_result = H[R[i][0] - 1][j] ^ H[R[i][1] - 1][j]
                if len(R[i]) > 2:
                    for k in range(2, len(R[i])):
                        xor_result = xor_result ^ H[R[i][k] - 1][j]
                        
                result[i].append(xor_result)
            
    return result
 




H = [[1, 1, 0, 0], 
     [0, 1, 1, 0],
     [0, 0, 1, 1]]

R = [[1], [2], [3], [1, 2], [2, 3], [1, 2, 3]]

result = matrix_to_vector_power(H, R)
print(result)

def Fk(m):
    return m


def Gkp(m):
    return m

def Gkp_inverse(m):
    return m

M = [1, 1, 0, 0]

def xor_gtm_hash(M, H, n):
    S = []
    t = len(H)
    m = len(H[0])
    
    for i in range(0, t):
        S.append([])
        for j in range (0, n):
            S[i].append(0)
        
    for j in range(0, m):
        Z = Fk(M[j])
        for i in range(0, t):
            if H[i][j] == 1:
                for k in range(0, n):
                    S[i][k] = S[i][k] ^ Z[k]
    
    return S

def xor_gtm_tag(M, H, n):
    t = len(H)
    m = len(H[0])
    T = []
    S = xor_gtm_hash(M, H)
    
    for i in range(0, t):
        T.append(Gkp(S[i]))

    return T


def xor_gtm_verify(Mp, Tp, H, n):
    
    Tc = xor_gtm_tag(Mp, H, n)
    tc = len(Tc)
    tp = len(Tp)
    t = 0
    
    if tc == tp:
        t = tc
    else:
        return False
    
    for i in range(0, t):
        for j in range(0, n):
            if Tc[i][j] != Tp[i][j]:
                return False
            
    return True


def xor_gtm_verify_S(Mp, Tp, H, n, R):
    t = len(H)
    m = len(H[0])
    v = len(R)
    
    Sp = []
    for i in range(0, t):
        Sp.append(Gkp_inverse(Tp[i]))
    
    Sc = xor_gtm_hash(Mp, H, n)
    SpR = matrix_to_vector_power(Sp, R)
    ScR = matrix_to_vector_power(Sc, R)
    
    B = []
    
    for i in range(0, v):
        checked = False
        for j in range(0, n):
            if SpR[i][j] != ScR[i][j]:
                B.append(False)
                checked = True
                break
        if checked == False:
            B.append(True)


    return B


def xor_gtm_detect(Mp, Tp, H, n, R):
    P = []
    
    for i in range(0, m):
        P.append(i)
        
    
    
    
    return P
    

[[1, 1, 0, 0], [0, 1, 1, 0], [0, 0, 1, 1], [1, 0, 1, 0], [0, 1, 0, 1], [1, 0, 0, 1]]
